In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import loadtxt
from xgboost import  XGBClassifier
from xgboost import plot_importance
import xgboost
from sklearn import metrics
import matplotlib.pyplot as plt
import platform
from matplotlib import font_manager, rc

plt.rcParams['axes.unicode_minus'] = False
if platform.system() == 'Darwin':    # 맥
    rc( 'font', family='AppleGothic' )
elif platform.system() == 'Windows': # 윈도우
    fontPath = 'c:/Windows/Fonts/malgun.ttf'
    fontName = font_manager.FontProperties( fname=fontPath ).get_name()
    rc( 'font', family=fontName )
else:
    print('알수없는 시스템. 미적용')
    
pd.set_option('display.expand_frame_repr', False)
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
final_dataset = pd.read_excel('./true.xlsx')

col = ['종가 (5일 평균)(원)_1일변화율',
       '변동성 (5일)_1일변화율', '순매수수량(외국인계)(20일합산)(주)_1일변화율', 'PER(IFRS-연결)_1일변화율',
       '순매수수량(개인)(20일합산)(주)_1일변화율', '종가 (60일 평균)(원)_1일변화율',
       '거래량 (20일 평균)(주)_1일변화율', '순매수수량(외국인계)(60일합산)(주)_1일변화율',
       '순매수수량(기관/외국인계)(60일합산)(주)_1일변화율', '순매수수량(외국인계)(주)_1일변화율',
       '순매수수량(기관계)(주)_1일변화율', '순매수수량(기관계)(60일합산)(주)_1일변화율',
       '순매수수량(개인)(주)_1일변화율', '외국인보유비중(티커)(%)_1일변화율', 'PSR(IFRS-연결)_1일변화율',
       '종가 (120일 평균)(원)_1일변화율', '종가 (20일 평균)(원)_1일변화율',
       '순매수수량(개인)(60일합산)(주)_1일변화율', '거래량 (60일 평균)(주)_1일변화율',
       'PCR(IFRS-연결)_1일변화율', '순매수수량(기관/외국인계)(20일합산)(주)_1일변화율',
       '순매수수량(기관계)(20일합산)(주)_1일변화율', '수익률 (1주)(%)_1일변화율', 'PBR(IFRS-연결)_1일변화율',
       '거래량 (5일 평균)(주)_1일변화율', '순매수수량(기관/외국인계)(주)_1일변화율',
       '종가 (5일 평균)(원)_2일변화율', '변동성 (5일)_2일변화율', '순매수수량(외국인계)(20일합산)(주)_2일변화율',
       'PER(IFRS-연결)_2일변화율', '순매수수량(개인)(20일합산)(주)_2일변화율',
       '종가 (60일 평균)(원)_2일변화율', '거래량 (20일 평균)(주)_2일변화율',
       '순매수수량(외국인계)(60일합산)(주)_2일변화율', '순매수수량(기관/외국인계)(60일합산)(주)_2일변화율',
       '순매수수량(외국인계)(주)_2일변화율', '순매수수량(기관계)(주)_2일변화율',
       '순매수수량(기관계)(60일합산)(주)_2일변화율', '순매수수량(개인)(주)_2일변화율',
       '외국인보유비중(티커)(%)_2일변화율', 'PSR(IFRS-연결)_2일변화율', '종가 (120일 평균)(원)_2일변화율',
       '종가 (20일 평균)(원)_2일변화율', '순매수수량(개인)(60일합산)(주)_2일변화율',
       '거래량 (60일 평균)(주)_2일변화율', 'PCR(IFRS-연결)_2일변화율',
       '순매수수량(기관/외국인계)(20일합산)(주)_2일변화율', '순매수수량(기관계)(20일합산)(주)_2일변화율',
       '수익률 (1주)(%)_2일변화율', 'PBR(IFRS-연결)_2일변화율', '거래량 (5일 평균)(주)_2일변화율',
       '순매수수량(기관/외국인계)(주)_2일변화율','수익률(%)(2)','Name']
data =  final_dataset[col]

In [ ]:
cols= ['변동성 (5일)_1일변화율',
 '순매수수량(외국인계)(20일합산)(주)_1일변화율',
 '순매수수량(기관계)(주)_1일변화율',
 '외국인보유비중(티커)(%)_1일변화율',
 '종가 (120일 평균)(원)_1일변화율',
 '거래량 (60일 평균)(주)_1일변화율',
 '순매수수량(기관/외국인계)(20일합산)(주)_1일변화율',
 '거래량 (5일 평균)(주)_1일변화율',
 '변동성 (5일)_2일변화율',
 '순매수수량(개인)(20일합산)(주)_2일변화율',
 '종가 (60일 평균)(원)_2일변화율',
 '거래량 (20일 평균)(주)_2일변화율',
 '순매수수량(외국인계)(60일합산)(주)_2일변화율',
 '외국인보유비중(티커)(%)_2일변화율',
 '순매수수량(개인)(60일합산)(주)_2일변화율',
 '순매수수량(기관계)(20일합산)(주)_2일변화율',
 '수익률 (1주)(%)_2일변화율',
 '순매수수량(기관/외국인계)(주)_2일변화율']

In [ ]:
def data_sp(data):
    com_name_set = list(set(data.Name))
    
    train_set = pd.DataFrame()
    test_set = pd.DataFrame()
    
    for i in range(len(com_name_set)):
        sub_set = data[ data.Name == com_name_set[i]]
        train_set = train_set.append(sub_set[:-60])
        test_set = test_set.append(sub_set[-60:])
    return train_set , test_set

train, test = data_sp(data)

In [ ]:
X_train = train[cols]
X_test = test[cols]

Y_train = train['수익률(%)(2)']
Y_test = test['수익률(%)(2)']

In [1]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

In [ ]:
Cs = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
gammas = [0.001, 0.01, 0.1, 1]
kernel = ['linear', 'rbf', 'sigmoid']#'precomputed','poly'
nfolds=5
criterion = ["gini", 'entropy']
#grid_search = GridSearchCV(SVC(), param_grid_svc, cv=nfolds)
n_estimators = [500,1000,1500]
learning_rate = [0.01]
max_depth = [4,5,6]
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
scoring = 'accuracy'
n_jobs= -1
penalty= ['l2']
max_features = [2]
linearscv_param_grid = {'C':Cs, 'penalty':penalty, 'random_state':random_state }
random_state = [4000]


In [ ]:
### 랜덤포레스트 튜닝

SVC_acc = []
SVC_pre = []
SVC_param = []
'C': Cs, 'gamma' : gammas,'kernel':kernel, 'random_state':random_state
for i in Cs:
    for b in kernel:
        for c in gammas:
            model = SVC(C = i,
                       kernel = b,
                       gamma = c,
                       random_state = random_state)

            model.fit(X_train ,Y_train)
            a = model.predict(X_test)
            t = metrics.confusion_matrix(Y_test,a)

            SCV_acc.append(metrics.accuracy_score(a,Y_test))
            SCV_pre.append(t[1,1]/t[:,1].sum())
            SCV_param.append(model)
                        
                        

In [ ]:
### 랜덤포레스트 튜닝

RF_acc = []
RF_pre = []
RF_param = []
for i in max_depth:
    for b in n_estimators:
        for c in criterion:
            model = RandomForestClassifier(max_depth = i,
                                           estimators = b,
                                           criterion = c)

            model.fit(X_train ,Y_train)
            a = model.predict(X_test)
            t = metrics.confusion_matrix(Y_test,a)

            RF_acc.append(metrics.accuracy_score(a,Y_test))
            RF_pre.append(t[1,1]/t[:,1].sum())
            RF_param.append(model)
                        
                        

In [ ]:
### 로지스틱 튜닝

LG_acc = []
LG_pre = []
LG_param = []

for i in Cs:
    model = RandomForestClassifier(C = i)

    model.fit(X_train ,Y_train)
    a = model.predict(X_test)
    t = metrics.confusion_matrix(Y_test,a)

    LG_acc.append(metrics.accuracy_score(a,Y_test))
    LG_pre.append(t[1,1]/t[:,1].sum())
    LG_param.append(model)

                        